## Before starting the code: Install Firefox on your local machine, login to Skype-Web using it, and accept all the cookies. Also, wait for the Skype Bing chat to appear on top of the chat list (if it already doesn't).

In [3]:
# !apt upgrade pip
!pip install selenium
!pip install termcolor
!pip install tqdm
!pip install pandas
# !pip install tqdm

!pip install openpyxl

### Importing Functions and Opening Skype-web

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
tqdm.pandas()
import traceback
import requests
import urllib.parse
import time
from time import gmtime, strftime
import os
from bs4 import BeautifulSoup
import re
import csv
from termcolor import colored
os.system('color')
from colorama import Fore, Back, Style
import sys
from selenium.webdriver import FirefoxOptions

chat_rate_limit = 20


base_url = "https://web.skype.com/?openPstnPage=true"
# firefox_user_path = "C:\\Users\\username\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\userprofile"
options=webdriver.FirefoxOptions()
options.profile = firefox_user_path
options.add_argument("--headless")
options.set_preference('profile', firefox_user_path)
driver = webdriver.Firefox(options=options)

# open Skype
driver.get(base_url)

# open Bing-AI chatbot
driver.implicitly_wait(20)
driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div[1]/div/div[1]/div[3]/div[1]/div[3]/div[2]/div[1]/div/div[1]/div/div/div[2]/div[3]/div/div').click()

<ipython-input-3-3c3399e18c1b>:31: DeprecationWarning: Setting a profile has been deprecated. Please use the set_preference and install_addons methods
  options.profile = firefox_user_path


### Initializing the response file

##### ALARM! Only run the following cell the "first time" that you are querying each chunk. Otherwise you will overwrite the file having all your previous queries lost!!!

In [4]:
# First Time

# ALARM! Only run this cell the first time that you are creating the file. Otherwise you will overwrite the file and your previous queries!!!

query_file_handle = open(query_file_path, 'w', encoding='UTF-8', newline='')
fieldnames = ['post_id','url','initial_claim','model','query','answer','created_at']
writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
writer.writeheader()
query_file_handle.close()

### Scraper Functions

In [4]:
model = 'BingAI'


fieldnames = ['post_id','url','initial_claim','model','query','answer','created_at']


def query_builder(text_x):
  if re.findall('[a-zA-Z0-9]$',text_x):
    text_x = text_x + '?'
  query_x = re.sub('\.$','?',text_x)
  query_x = re.sub('!$','?',query_x)
  return query_x

def get_link_or_text(webdriver_element_x):
    soup=BeautifulSoup(webdriver_element_x.get_attribute('innerHTML'), "html.parser")
    try:
        return soup.find('a').get('href')
    except:
        return soup.text
    
def merge_link_and_text(webdriver_element_x):
    soup=BeautifulSoup(webdriver_element_x.get_attribute('innerHTML'), "html.parser")
    text = ''
    refs_htmls = soup.find_all('div')
    for i,refs_html in enumerate(refs_htmls):
#         print(i)
        try:
            text = text + refs_html.previous_sibling + ' '
        
            text = text + refs_html.find('a').get('href') + ' '
        except:
            text = text + refs_html.text + ' '
        if i == len(refs_html) -1 :
            text = text + refs_html.next_sibling
    return text

def query_BingAI(query_text):
    global t
    print(t)
    # Check if daily limit is reached
    all_responses_ = driver.find_elements(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[2]/div/div/div[1]/div[1]/div[2]/div[1]/div/div/div')
    if all_responses_!=[]:
        last_response_ = all_responses_[-2]
        if last_response_.text == "Sorry, you've reached your daily limit to chat. Let's pick up again tomorrow!":
            print('daily limit reached!')
            sys.exit()

    # Handling the 20 chat per session limit
    if t % (chat_rate_limit-3) == 0:
        driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div[2]/div/div/div/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div').send_keys('newtopic')
        # click on send button
        driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div[1]/div[2]/div[2]/div/div/div/div[2]/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/button/div/div').click()

        print('Vahid: moving to a new chat session ...')
        time.sleep(20)
        
    while True:    
    
        t=t+1


        # type the prompt
        driver.implicitly_wait(20)
        time.sleep(1)
        driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div[2]/div/div/div/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div').send_keys(query_text)

        # click on send button
        driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div[1]/div[2]/div[2]/div/div/div/div[2]/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/button/div/div').click()

        # collect the response
        time.sleep(40)
    #     driver.implicitly_wait(20)
        creation_time = strftime("%Y-%m-%d %H:%M:%S", gmtime())
        all_responses = driver.find_elements(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div/div/div[1]/div/div[1]/div[1]/div/div/div[1]/div/div[2]/div/div')
        last_response = all_responses[-2]
        
        last_response_partitions = last_response.find_elements(By.XPATH,'.//div[1]/div[1]/div[2]/div[1]/div/div/div/div/div[1]/div')

        #  the XPath of a response with links and a response without link is different. The following checks for that                                              
        if last_response_partitions:
            try:
#                 response_text = " ".join([get_link_or_text(x) for x in last_response_partitions])
                response_text = merge_link_and_text(last_response_partitions[0])
            except Exception as e:
                response_text = ''
                print('Vahid: something went wrong. trying again')
                print(e)
        else:
            try:
                response_text = last_response.find_element(By.XPATH,'.//div[1]/div[1]/div[2]/div[1]/div/div/div/div').text
            except Exception as e:
                response_text = ''
                print('Vahid: something went wrong. trying again')
                print(e)
        if (response_text != '(sad)') and (response_text != '(smileeyes)') and (response_text != '') and (response_text != "I'm sorry but I prefer not to continue this conversation. I'm still learning so I appreciate your understanding and patience."):
            break

    return response_text, creation_time
    
def get_chatbot_response(query_text_x, model_x):
    
  if model_x == 'BingAI':
    response_text, creation_time = query_BingAI(query_text_x)

  elif (model_x.split('-')[1] == '3.5') or (model_x.split('-')[1] == '4'):

    completion = openai.ChatCompletion.create(model = model_x, 
                                          messages=[{"role": "user", "content": query_text_x}], 
                                          max_tokens = 1024,
                                          temperature = 0)
    response_text = completion["choices"][0]["message"]["content"]
    creation_time = completion['created']

    
  print(Fore.BLUE,f'Response:    ',end='')
  print(Fore.BLACK, response_text)
  return response_text, creation_time

def get_query_and_write(post_id_x, text_x, url_x):

  if post_id_x in already_queried_posts:
    print(f'{str(post_id_x)} is already queried')
    return None

  else:
    query_x = query_builder(text_x)
    print(Fore.RED,f'Prompt:    ',end='')
    print(Fore.BLACK, query_x)
    response_text, creation_time = get_chatbot_response(query_text_x=query_x, model_x=model)

    query_file_handle = open(query_file_path, 'a', newline='', encoding='UTF-8')
    writer = csv.DictWriter(query_file_handle, fieldnames=fieldnames)
    writer.writerow({'post_id':post_id_x,'url':url_x,'initial_claim':text_x,'model':model,'query':query_x,'answer':response_text,'created_at':creation_time})
    query_file_handle.close()
    print(Fore.GREEN,'\ndone\n')

#     time.sleep(3)

### The main function

In [6]:
t=1
driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div[1]/div/div[2]/div/div/div/div[2]/div/div/div/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div[2]/div/div/div/div[2]/div/div/div').send_keys('newtopic')
# click on send button
driver.find_element(By.XPATH,'/html/body/div[1]/div/div/div[1]/div[2]/div/div/div[1]/div[2]/div[2]/div/div/div/div[2]/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/button/div/div').click()

time.sleep(10)

meta_data = pd.read_excel(f'Kialo_MetaData_popular_chunk_{chunk_number}.xlsx')
query_file_df = pd.read_csv(query_file_path , encoding='UTF-8')
already_queried_posts = list(query_file_df['post_id'])

meta_data.progress_apply(lambda x: get_query_and_write(post_id_x=x['post_id'],
                                                       text_x=x['initial_claim'],
                                                       url_x=x['url']),axis=1)

  0%|          | 0/97 [00:00<?, ?it/s]

 Prompt:     Germany should dissolve its armed forces ("Bundeswehr")?
1
 Response:     Germany's military has been a topic of debate for years. While some people believe that Germany should dissolve its armed forces, others believe that it should not. In 2011, Germany abolished conscription and is now concentrating on smaller specialist forces  https://www.bbc.com/news/world-europe-40172317 . However, Chancellor Olaf Scholz's plan to boost defense spending could remake Germany's beleaguered military into the strongest armed forces in Europe . However, Chancellor Olaf Scholz's plan to boost defense spending could remake Germany's beleaguered military into the strongest armed forces in Europe  https://www.npr.org/2022/03/22/1087859567/germany-military-buildup-russia-invasion-ukraine 
 
done

 Prompt:     The sale of semi-automatic rifles (external box magazine) should be banned?
2
 Response:     The sale of semi-automatic rifles with external box magazines has been a topic of debate for 

 Response:     According to an article on the Brookings Institution website, transparency is an important aspect of democratic governance. Transparency can help to ensure that public officials are held accountable for their actions and that the public has access to information that is necessary to make informed decisions. However, there are some concerns about the extent to which transparency should be required. Some argue that too much transparency can be counterproductive and that it can lead to a lack of trust between public officials and the public. Others argue that transparency is essential for ensuring that public officials are held accountable for their actions. Is there anything else I can help you with?
 
done

 Prompt:     Laws should forbid web sites from requiring new locally-stored, single-site, password-based authentication?
14
 Response:     According to an article on the Electronic Frontier Foundation website, password-based authentication is a common method of securin

KeyboardInterrupt: 